### RNN learns to add

In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np

Using TensorFlow backend.


Class Sequences for input and output

In [2]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.

        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.

        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


Class colors (for nicer output)

In [3]:
class colors:
    ok = "\033[1;32m"
    fail = "\033[1;31m"
    close = "\033[39;49m"

Program global constants and variables

In [4]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

    Prepare data for training

In [5]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):  
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)


Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


The RNN Network

In [6]:
# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


Train model and show predictions

In [7]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Instructions for updating:
Use tf.cast instead.
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 56s 1ms/step - loss: 1.8834 - acc: 0.3225 - val_loss: 1.7817 - val_acc: 0.3403
Q 661+20  T 681  ☒ 161 
Q 90+155  T 245  ☒ 104 
Q 42+244  T 286  ☒ 144 
Q 421+706 T 1127 ☒ 114 
Q 53+71   T 124  ☒ 64  
Q 237+92  T 329  ☒ 144 
Q 19+956  T 975  ☒ 104 
Q 345+856 T 1201 ☒ 114 
Q 94+44   T 138  ☒ 144 
Q 907+77  T 984  ☒ 104 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 44s 972us/step - loss: 1.7280 - acc: 0.3619 - val_loss: 1.6720 - val_acc: 0.3805
Q 64+46   T 110  ☒ 550 
Q 114+77  T 191  ☒ 599 
Q 40+14   T 54   ☒ 14  
Q 849+1   T 850  ☒ 109 
Q 590+164 T 754  ☒ 101 
Q 40+454  T 494  ☒ 559 
Q 379+844 T 1223 ☒ 1319
Q 429+673 T 1102 ☒ 1319
Q 419+156 T 575  ☒ 1

Q 66+763  T 829  ☑ 829 
Q 92+945  T 1037 ☑ 1037
Q 495+862 T 1357 ☒ 1366
Q 301+356 T 657  ☒ 656 
Q 49+866  T 915  ☒ 914 
Q 460+881 T 1341 ☒ 1342
Q 827+41  T 868  ☑ 868 
Q 320+148 T 468  ☑ 468 
Q 139+637 T 776  ☑ 776 
Q 2+141   T 143  ☒ 144 

--------------------------------------------------
Iteration 14
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 33s 735us/step - loss: 0.3381 - acc: 0.8989 - val_loss: 0.2744 - val_acc: 0.9300
Q 6+307   T 313  ☑ 313 
Q 218+83  T 301  ☑ 301 
Q 9+923   T 932  ☑ 932 
Q 477+207 T 684  ☑ 684 
Q 50+38   T 88   ☑ 88  
Q 486+666 T 1152 ☒ 1142
Q 788+5   T 793  ☑ 793 
Q 836+38  T 874  ☑ 874 
Q 60+850  T 910  ☒ 911 
Q 164+1   T 165  ☑ 165 

--------------------------------------------------
Iteration 15
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 33s 741us/step - loss: 0.2265 - acc: 0.9475 - val_loss: 0.1960 - val_acc: 0.9537
Q 66+13   T 79  

45000/45000 [==============================] - 33s 736us/step - loss: 0.0150 - acc: 0.9992 - val_loss: 0.0180 - val_acc: 0.9972
Q 3+295   T 298  ☑ 298 
Q 44+71   T 115  ☑ 115 
Q 44+369  T 413  ☑ 413 
Q 8+735   T 743  ☑ 743 
Q 369+35  T 404  ☑ 404 
Q 683+724 T 1407 ☑ 1407
Q 519+8   T 527  ☑ 527 
Q 315+47  T 362  ☑ 362 
Q 4+206   T 210  ☑ 210 
Q 26+130  T 156  ☑ 156 

--------------------------------------------------
Iteration 27
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 33s 730us/step - loss: 0.0123 - acc: 0.9994 - val_loss: 0.0156 - val_acc: 0.9978
Q 29+66   T 95   ☑ 95  
Q 653+81  T 734  ☑ 734 
Q 69+997  T 1066 ☑ 1066
Q 5+477   T 482  ☑ 482 
Q 49+563  T 612  ☑ 612 
Q 542+22  T 564  ☑ 564 
Q 449+328 T 777  ☑ 777 
Q 955+4   T 959  ☑ 959 
Q 883+982 T 1865 ☑ 1865
Q 107+255 T 362  ☑ 362 

--------------------------------------------------
Iteration 28
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 33s 736us/step - loss: 0.0038 - acc: 0.9999 - val_loss: 0.0054 - val_acc: 0.9994
Q 33+16   T 49   ☑ 49  
Q 642+629 T 1271 ☑ 1271
Q 861+451 T 1312 ☑ 1312
Q 128+72  T 200  ☑ 200 
Q 46+530  T 576  ☑ 576 
Q 542+64  T 606  ☑ 606 
Q 856+77  T 933  ☑ 933 
Q 14+8    T 22   ☑ 22  
Q 809+80  T 889  ☑ 889 
Q 97+30   T 127  ☑ 127 

--------------------------------------------------
Iteration 40
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 34s 751us/step - loss: 0.0030 - acc: 1.0000 - val_loss: 0.0050 - val_acc: 0.9992
Q 599+66  T 665  ☑ 665 
Q 650+100 T 750  ☒ 740 
Q 510+426 T 936  ☑ 936 
Q 2+791   T 793  ☑ 793 
Q 85+862  T 947  ☑ 947 
Q 592+4   T 596  ☑ 596 
Q 21+374  T 395  ☑ 395 
Q 0+353   T 353  ☑ 353 
Q 540+14  T 554  ☑ 554 
Q 44+159  T 203  ☑ 203 

--------------------------------------------------
Iteration 41
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 34s 752us/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.0030 - val_acc: 0.9994
Q 5+398   T 403  ☑ 403 
Q 253+66  T 319  ☑ 319 
Q 735+72  T 807  ☑ 807 
Q 69+239  T 308  ☑ 308 
Q 818+54  T 872  ☑ 872 
Q 97+708  T 805  ☑ 805 
Q 51+589  T 640  ☑ 640 
Q 81+33   T 114  ☑ 114 
Q 841+98  T 939  ☑ 939 
Q 998+119 T 1117 ☑ 1117

--------------------------------------------------
Iteration 53
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 35s 777us/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.0026 - val_acc: 0.9996
Q 732+813 T 1545 ☑ 1545
Q 94+494  T 588  ☑ 588 
Q 62+57   T 119  ☑ 119 
Q 280+21  T 301  ☑ 301 
Q 466+513 T 979  ☑ 979 
Q 689+40  T 729  ☑ 729 
Q 828+88  T 916  ☑ 916 
Q 387+958 T 1345 ☑ 1345
Q 56+90   T 146  ☑ 146 
Q 448+177 T 625  ☑ 625 

--------------------------------------------------
Iteration 54
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

...

45000/45000 [==============================] - 36s 791us/step - loss: 0.0308 - acc: 0.9910 - val_loss: 0.0394 - val_acc: 0.9874
Q 912+2   T 914  ☑ 914 
Q 652+724 T 1376 ☑ 1376
Q 31+94   T 125  ☑ 125 
Q 116+735 T 851  ☑ 851 
Q 335+970 T 1305 ☑ 1305
Q 7+266   T 273  ☑ 273 
Q 33+807  T 840  ☑ 840 
Q 899+579 T 1478 ☑ 1478
Q 713+62  T 775  ☒ 774 
Q 123+332 T 455  ☑ 455 

--------------------------------------------------
Iteration 144
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 37s 811us/step - loss: 0.0053 - acc: 0.9986 - val_loss: 0.0019 - val_acc: 0.9995
Q 804+88  T 892  ☑ 892 
Q 179+151 T 330  ☑ 330 
Q 309+73  T 382  ☑ 382 
Q 319+361 T 680  ☑ 680 
Q 436+76  T 512  ☑ 512 
Q 680+829 T 1509 ☑ 1509
Q 547+741 T 1288 ☑ 1288
Q 382+880 T 1262 ☑ 1262
Q 34+449  T 483  ☑ 483 
Q 16+356  T 372  ☑ 372 

--------------------------------------------------
Iteration 145
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=

45000/45000 [==============================] - 34s 758us/step - loss: 6.6552e-04 - acc: 0.9999 - val_loss: 0.0012 - val_acc: 0.9998
Q 384+95  T 479  ☑ 479 
Q 55+246  T 301  ☑ 301 
Q 14+904  T 918  ☑ 918 
Q 743+240 T 983  ☑ 983 
Q 421+4   T 425  ☑ 425 
Q 65+390  T 455  ☑ 455 
Q 66+930  T 996  ☑ 996 
Q 90+922  T 1012 ☑ 1012
Q 71+446  T 517  ☑ 517 
Q 4+232   T 236  ☑ 236 

--------------------------------------------------
Iteration 157
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 33s 739us/step - loss: 3.9682e-04 - acc: 1.0000 - val_loss: 0.0010 - val_acc: 0.9998
Q 30+485  T 515  ☑ 515 
Q 814+534 T 1348 ☑ 1348
Q 464+78  T 542  ☑ 542 
Q 2+60    T 62   ☑ 62  
Q 956+43  T 999  ☑ 999 
Q 805+29  T 834  ☑ 834 
Q 75+327  T 402  ☑ 402 
Q 968+31  T 999  ☑ 999 
Q 912+2   T 914  ☑ 914 
Q 327+714 T 1041 ☑ 1041

--------------------------------------------------
Iteration 158
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/

45000/45000 [==============================] - 38s 836us/step - loss: 2.9133e-04 - acc: 1.0000 - val_loss: 0.0010 - val_acc: 0.9998
Q 680+802 T 1482 ☑ 1482
Q 554+361 T 915  ☑ 915 
Q 14+802  T 816  ☑ 816 
Q 604+531 T 1135 ☑ 1135
Q 485+521 T 1006 ☑ 1006
Q 943+1   T 944  ☑ 944 
Q 984+390 T 1374 ☑ 1374
Q 0+700   T 700  ☑ 700 
Q 446+930 T 1376 ☑ 1376
Q 67+910  T 977  ☑ 977 

--------------------------------------------------
Iteration 170
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 37s 813us/step - loss: 2.3707e-04 - acc: 1.0000 - val_loss: 8.7843e-04 - val_acc: 0.9998
Q 67+836  T 903  ☑ 903 
Q 72+271  T 343  ☑ 343 
Q 5+374   T 379  ☑ 379 
Q 55+186  T 241  ☑ 241 
Q 89+422  T 511  ☑ 511 
Q 12+797  T 809  ☑ 809 
Q 538+407 T 945  ☑ 945 
Q 120+53  T 173  ☑ 173 
Q 994+958 T 1952 ☑ 1952
Q 966+2   T 968  ☑ 968 

--------------------------------------------------
Iteration 171
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45

45000/45000 [==============================] - 33s 740us/step - loss: 3.9934e-04 - acc: 1.0000 - val_loss: 0.0011 - val_acc: 0.9998
Q 78+275  T 353  ☑ 353 
Q 33+768  T 801  ☑ 801 
Q 0+657   T 657  ☑ 657 
Q 28+89   T 117  ☑ 117 
Q 6+299   T 305  ☑ 305 
Q 37+925  T 962  ☑ 962 
Q 595+50  T 645  ☑ 645 
Q 27+620  T 647  ☑ 647 
Q 103+60  T 163  ☑ 163 
Q 226+42  T 268  ☑ 268 

--------------------------------------------------
Iteration 183
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 35s 772us/step - loss: 2.5126e-04 - acc: 1.0000 - val_loss: 9.4935e-04 - val_acc: 0.9998
Q 783+39  T 822  ☑ 822 
Q 82+205  T 287  ☑ 287 
Q 27+897  T 924  ☑ 924 
Q 657+755 T 1412 ☑ 1412
Q 455+7   T 462  ☑ 462 
Q 753+44  T 797  ☑ 797 
Q 78+322  T 400  ☑ 400 
Q 283+461 T 744  ☑ 744 
Q 19+34   T 53   ☑ 53  
Q 440+94  T 534  ☑ 534 

--------------------------------------------------
Iteration 184
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45

45000/45000 [==============================] - 33s 731us/step - loss: 6.3796e-05 - acc: 1.0000 - val_loss: 5.6201e-04 - val_acc: 0.9998
Q 252+293 T 545  ☑ 545 
Q 657+180 T 837  ☑ 837 
Q 938+98  T 1036 ☑ 1036
Q 15+751  T 766  ☑ 766 
Q 311+266 T 577  ☑ 577 
Q 373+92  T 465  ☑ 465 
Q 214+59  T 273  ☑ 273 
Q 98+912  T 1010 ☑ 1010
Q 9+72    T 81   ☑ 81  
Q 800+8   T 808  ☑ 808 

--------------------------------------------------
Iteration 196
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 34s 760us/step - loss: 0.0396 - acc: 0.9898 - val_loss: 0.0036 - val_acc: 0.9990
Q 9+792   T 801  ☑ 801 
Q 687+22  T 709  ☑ 709 
Q 135+997 T 1132 ☑ 1132
Q 71+13   T 84   ☑ 84  
Q 846+3   T 849  ☑ 849 
Q 94+313  T 407  ☑ 407 
Q 4+178   T 182  ☑ 182 
Q 917+28  T 945  ☑ 945 
Q 86+239  T 325  ☑ 325 
Q 7+309   T 316  ☑ 316 

--------------------------------------------------
Iteration 197
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/